In [2]:
# 팀과 타율율

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
driver = webdriver.Chrome()
driver.get("https://www.koreabaseball.com/Record/Team/Hitter/Basic1.aspx")

time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup.find('table', class_='tData tt')

team_data = []

for row in table.select('tbody > tr'):
    cols = row.find_all('td')
    if len(cols) > 1:
        rank = cols[0].text.strip()
        team = cols[1].text.strip()
        avg = cols[2].text.strip()
        
        team_info = {
            '순위': rank,
            '팀명': team,
            '타율': avg
        }
        team_data.append(team_info)

driver.quit()

df = pd.DataFrame(team_data)

df.to_csv('data/kbo_team_avg.csv', index=False)

In [15]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


driver = webdriver.Chrome()
driver.get("https://www.koreabaseball.com/Record/TeamRank/TeamRank.aspx")

wait = WebDriverWait(driver, 10)

year_dropdown = wait.until(EC.presence_of_element_located((By.ID, "cphContents_cphContents_cphContents_ddlYear")))
select = Select(year_dropdown)
select.select_by_visible_text("2024")

time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'html.parser')
vs_team_div = soup.find("div", id="cphContents_cphContents_cphContents_pnlVsTeam")
table = vs_team_div.find('table', class_='tData')

team_data = []

for row in table.select('tbody > tr'):
    cols = row.find_all('td')
    if len(cols) > 1:
        team = cols[0].text.strip()
        kia = cols[1].text.strip()
        samsung = cols[2].text.strip()
        lg = cols[3].text.strip()
        doosan = cols[4].text.strip()
        kt = cols[5].text.strip()
        ssg = cols[6].text.strip()
        lotte = cols[7].text.strip()
        hanwha = cols[8].text.strip()
        nc = cols[9].text.strip()
        kiwoom = cols[10].text.strip()
        summary = cols[11].text.strip()
        
        team_info = {
            '팀명': team,
            'KIA': kia,
            '삼성': samsung,
            'LG': lg,
            '두산': doosan,
            'KT': kt,
            'SSG': ssg,
            '롯데': lotte,
            '한화': hanwha,
            'NC': nc,
            '키움': kiwoom,
            '합계': summary
        }
        print(team_info)
        team_data.append(team_info)

driver.quit()

df = pd.DataFrame(team_data)

df.to_csv('data/team_last_year.csv', index=False)

{'팀명': 'KIA', 'KIA': '■', '삼성': '12-4-0', 'LG': '13-3-0', '두산': '6-9-1', 'KT': '9-7-0', 'SSG': '6-10-0', '롯데': '7-8-1', '한화': '11-5-0', 'NC': '12-4-0', '키움': '11-5-0', '합계': '87-55-2'}
{'팀명': '삼성', 'KIA': '4-12-0', '삼성': '■', 'LG': '8-7-1', '두산': '12-4-0', 'KT': '7-8-1', 'SSG': '7-9-0', '롯데': '10-6-0', '한화': '10-6-0', 'NC': '10-6-0', '키움': '10-6-0', '합계': '78-64-2'}
{'팀명': 'LG', 'KIA': '3-13-0', '삼성': '7-8-1', 'LG': '■', '두산': '9-7-0', 'KT': '9-7-0', 'SSG': '11-4-1', '롯데': '11-5-0', '한화': '8-8-0', 'NC': '12-4-0', '키움': '6-10-0', '합계': '76-66-2'}
{'팀명': '두산', 'KIA': '9-6-1', '삼성': '4-12-0', 'LG': '7-9-0', '두산': '■', 'KT': '12-4-0', 'SSG': '7-9-0', '롯데': '7-8-1', '한화': '6-10-0', 'NC': '12-4-0', '키움': '10-6-0', '합계': '74-68-2'}
{'팀명': 'KT', 'KIA': '7-9-0', '삼성': '8-7-1', 'LG': '7-9-0', '두산': '4-12-0', 'KT': '■', 'SSG': '8-8-0', '롯데': '8-7-1', '한화': '6-10-0', 'NC': '10-6-0', '키움': '14-2-0', '합계': '72-70-2'}
{'팀명': 'SSG', 'KIA': '10-6-0', '삼성': '9-7-0', 'LG': '4-11-1', '두산': '9-7-0', 'KT': 

In [22]:
df_team = pd.read_csv('data/team_last_year.csv')

df_team = df_team.drop(columns=['합계'])

team_dict = []

for idx, row in df_team.iterrows():
    team_name = row['팀명']
    team_data = {'팀명': team_name}

    for col in df_team.columns[1:]:  # 팀명을 제외한 상대 팀들
        value = row[col]
        if '-' in str(value):
            win, lose, draw = map(int, value.split('-'))
            win_rate = int(win) / int(win + lose + draw)
            team_data[col] = win_rate
        else:
            team_data[col] = '■'  # 자기 자신은 '■'로 표시되므로 제외하거나 None 처리

    team_dict.append(team_data)
    
df = pd.DataFrame(team_dict)

df.to_csv('data/last_year_win_rate.csv', index=False)